In [2]:
import pandas as pd

### 크롤링

# Dart 크롤링

In [5]:
import pandas as pd
#한국거래소에서 종목 코드 받아옵니다. [0]은 헤더를 첫번째 행으로 지정하기 위해 사용 했습니다.
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13')[0]

In [6]:
code_df

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,AJ네트웍스,95570,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,손삼달,http://www.ajnet.co.kr,서울특별시
1,BNK금융지주,138930,기타 금융업,금융지주회사,2011-03-30,12월,빈대인,http://www.bnkfg.com,부산광역시
2,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시
3,GS,78930,기타 금융업,지주회사/부동산 임대,2004-08-05,12월,"허태수, 홍순기 (각자 대표이사)",NaN,서울특별시
4,HDC현대산업개발,294870,건물 건설업,"외주주택, 자체공사, 일반건축, 토목 등",2018-06-12,12월,"최익훈, 정익희, 김회언 (각자 대표이사)",http://www.hdc-dvp.com,서울특별시
...,...,...,...,...,...,...,...,...,...
2617,카이바이오텍,446600,의약품 제조업,방사성 진단 및 치료 의약품,2022-12-23,12월,김영덕,http://www.kaibiotech.com/,전라북도
2618,코스텍시스템,169670,특수 목적용 기계 제조업,"반도체 웨이퍼 이송장비, 본딩장비",2022-01-21,12월,배준호,http://www.kosteks.com/,경기도
2619,타임기술,318660,소프트웨어 개발 및 공급업,"ILS(종합군수지원), IETM(전자식기술교범), CBT(전자식교보재), 기술번역",2021-06-24,12월,주양효,http://timett.co.kr,경상남도
2620,테크엔,308700,전구 및 조명장치 제조업,LED조명장치,2018-12-21,12월,"이지원,박철(각자 대표이사)",http://www.techen.co.kr,대구광역시


In [2]:
# 다트 사업보고서 selenium 사용해 크롤링
from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup, Tag
from selenium.webdriver.common.keys import Keys
import time
import random
from openpyxl import Workbook
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from collections import defaultdict

options = Options()
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
options.add_experimental_option('excludeSwitches', ['enable-logging']) # Usb Error ignore
options.add_argument("no-sandbox") 
options.add_argument('window-size=1920x1080')
options.add_argument('headless')
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('user-agent='+ user_agent)
# options.add_argument('--proxy-server=socks5://127.0.0.1:9050')
driver = webdriver.Chrome(options = options)
company_names = []  # 회사명을 저장하는 리스트
company_ticker = [] # 종목코드를 저장하는 리스트
company_industries = [] # 업종을 저장하는 리스트
company_descriptions = []  # Description을 저장하는 리스트
i = 1
for index, row in code_df.iterrows():
    # if i == 3: break
    try:
        ticker = str(row['종목코드']).zfill(6)  # 종목코드 6자리로 맞춰주기
        driver.get(f"https://dart.fss.or.kr/dsab007/main.do?option=corp")
        search_input = driver.find_element(By.CSS_SELECTOR, '#textCrpNm')
        # driver.find_element(By.CSS_SELECTOR, '#btnPlus').click() # 상세조건 열기
        # driver.find_element(By.CSS_SELECTOR, '#corporationType > option:nth-child(2)').click() # 법인유형 유가증권시장 선택
        search_input.send_keys(ticker)

        date_range = driver.find_element(By.CSS_SELECTOR, '#date7').click()
        driver.find_element(By.CSS_SELECTOR, '#li_01 > label:nth-child(1) > img').click() # 정기공시 체크박스 클릭
        driver.find_element(By.CSS_SELECTOR, '#li_01').click() # 정기공시 창 클릭
        driver.find_element(By.CSS_SELECTOR, '#totalChoice_01').click() # 전체선택 해제
        driver.find_element(By.CSS_SELECTOR, '#publicTypeDetail_A001').click() # 사업보고서 체크박스 선택
        driver.find_element(By.CSS_SELECTOR, '#searchForm > div.subSearchWrap > div.btnArea > a.btnSearch').click() # 검색 클릭
        time.sleep(1)
        tbody_list = driver.find_elements(By.CSS_SELECTOR, '#tbody')
        # href = td.get_attribute('href')

        # print(href)
        for tr in tbody_list:
            td = tr.find_elements(By.CLASS_NAME, 'tL')[1]
            a_tag = td.find_element(By.TAG_NAME, 'a')
            href = a_tag.get_attribute('href')
            break

        driver.get(href)
        info = driver.find_element(By.ID, '10_anchor') # 
        if (info.text == "5. 정관에 관한 사항"):
            info = driver.find_element(By.ID, '12_anchor')
        info.click()
        driver.switch_to.frame("ifrm")
        body = driver.find_element(By.CSS_SELECTOR, 'body')
        p_str = ''
        # p_tag = body.find_elements(By.TAG_NAME, 'p')
        immediate_children = driver.find_elements(By.XPATH ,"/html/body/*")
        # child_elements = body.find_elements(By.XPATH , ".//*")
        paragraphs = [child.text for child in immediate_children]
        # paragraphs = []
        # for child in immediate_children:
        #     if child.tag_name == "table":
        #         child_td = child.find_elements(By.TAG_NAME, 'td')
        #         for td in child_td:
        #             paragraphs.append(td.text)
        #         continue
        #     paragraphs.append(child.text)
            


        # print(paragraphs)
        company_names.append(row['회사명'])  # 회사명을 리스트에 추가
        company_ticker.append(row['종목코드']) # 종목코드를 리스트에 추가
        company_industries.append(row['업종']) # 업종을 리스트에 추가
        company_descriptions.append(paragraphs)  # Description을 리스트에 추가
        
        print(f"{row['회사명']} 완료 {len(code_df)-i} 번 남았습니다")
        i += 1
    except:
        print(f"{row['회사명']} 존재x {len(code_df)-i} 번 남았습니다")
        i += 1
        continue
# 엑셀 파일 생성
wb = Workbook()
ws = wb.active

ws.cell(row=1, column=1, value='Company')
ws.cell(row=1, column=2, value='Ticker')
ws.cell(row=1, column=3, value='Industry')
ws.cell(row=1, column=4, value='Description')
# Description을 각각의 셀에 저장
for i, (name, paragraphs) in enumerate(zip(company_names, company_descriptions)):
    for j, p_text in enumerate(paragraphs):
        # print(p_text)
        ws.cell(row=i+2, column=j+3, value=p_text)

    # 회사명 저장
    print("회사명: ", name)
    ws.cell(row=i+2, column=1, value=name)
    # 종목코드 저장
    ws.cell(row=i+2, column=2, value=company_ticker[i])
    # 업종 저장
    ws.cell(row=i+2, column=3, value=company_industries[i])
    
wb.save('dart_사업개요(코스피, 코스닥).csv')
# 드라이버 종료
driver.quit()

AJ네트웍스 완료 2620 번 남았습니다
BNK금융지주 완료 2619 번 남았습니다
DSR 완료 2618 번 남았습니다
GS 완료 2617 번 남았습니다
HDC현대산업개발 완료 2616 번 남았습니다
HD한국조선해양 완료 2615 번 남았습니다
KC그린홀딩스 존재x 2614 번 남았습니다
KG케미칼 완료 2613 번 남았습니다
KH 필룩스 완료 2612 번 남았습니다
KPX케미칼 완료 2611 번 남았습니다
KTis 완료 2610 번 남았습니다
LG이노텍 완료 2609 번 남았습니다
LG헬로비전 완료 2608 번 남았습니다
LX홀딩스 완료 2607 번 남았습니다
MH에탄올 완료 2606 번 남았습니다
SK이노베이션 완료 2605 번 남았습니다
SK케미칼 완료 2604 번 남았습니다
SNT다이내믹스 완료 2603 번 남았습니다
STX 완료 2602 번 남았습니다
SUN&L 완료 2601 번 남았습니다
갤럭시아에스엠 완료 2600 번 남았습니다
경농 완료 2599 번 남았습니다
경동인베스트 완료 2598 번 남았습니다
광전자 완료 2597 번 남았습니다
국도화학 완료 2596 번 남았습니다
국보 완료 2595 번 남았습니다
극동유화 완료 2594 번 남았습니다
금양 완료 2593 번 남았습니다
노루페인트 완료 2592 번 남았습니다
녹십자홀딩스 완료 2591 번 남았습니다
대덕전자 완료 2590 번 남았습니다
대림통상 완료 2589 번 남았습니다
대성산업 완료 2588 번 남았습니다
더블유게임즈 완료 2587 번 남았습니다
동국홀딩스 완료 2586 번 남았습니다
두산퓨얼셀 완료 2585 번 남았습니다
롯데정보통신 완료 2584 번 남았습니다
롯데지주 완료 2583 번 남았습니다
메타랩스 완료 2582 번 남았습니다
명문제약 완료 2581 번 남았습니다
모나리자 완료 2580 번 남았습니다
미원화학 완료 2579 번 남았습니다
보령 완료 2578 번 남았습니다
부국증권 완료 2577 번 남았습니다
비케이탑스 완료 2576 번 남았습니다
삼부토건 완료 2575 번 남